In [0]:
from pyspark.sql.functions import col, explode, arrays_zip, from_unixtime,date_format,round

In [0]:
df_bronze = spark.table('stocks.bronze_dev.stocks')

In [0]:
df_silver = (df_bronze
    .selectExpr("chart.result[0].meta.symbol as symbol",
                "chart.result[0].meta.fiftyTwoWeekHigh as fiftyTwoWeekHigh",
                "chart.result[0].meta.fiftyTwoWeekLow as fiftyTwoWeekLow",
                "chart.result[0].meta.longName as stockName",
                "chart.result[0].timestamp as timestamp",
                "chart.result[0].indicators.quote[0].open as open",
                "chart.result[0].indicators.quote[0].high as high",
                "chart.result[0].indicators.quote[0].low as low",
                "chart.result[0].indicators.quote[0].close as close",
                "chart.result[0].indicators.quote[0].volume as volume",
                "chart.result[0].indicators.adjclose[0].adjclose as adjclose")
    # Zip arrays together so each index aligns
    .withColumn("zipped", arrays_zip("timestamp","open","high","low","close","volume","adjclose"))
    # Explode zipped array → one row per trading day
    .withColumn("zipped", explode(col("zipped")))
    .withColumn("trade_date",date_format(col("zipped.timestamp").cast("timestamp"),"yyyy-MM-dd"))
    # Flatten zipped struct into columns
    .select(
        col("symbol"),
        col("stockName"),
        col("trade_date").cast("date"),
        round(col("zipped.open"),2).alias("open"),
        round(col("zipped.high"),2).alias("high"),
        round(col("zipped.low"),2).alias("low"),
        round(col("zipped.close"),2).alias("close"),
        col("zipped.volume").alias("volume"),
        round(col("zipped.adjclose"),2).alias("adjclose"),
        col("fiftyTwoWeekHigh"),
        col("fiftyTwoWeekLow")
    )
    
)

In [0]:
df_silver.createOrReplaceTempView("silver_view")

In [0]:
%sql
create or replace table stocks.silver_dev.stocks as
select * from silver_view